<a href="https://colab.research.google.com/github/lapestand/bpe/blob/master/byte_pair_encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Byte Pair Encoding



## Train Data

In [1]:
test_corpus = """
    Yeni bir dil öğrenmek, zorlu ancak ödüllendirici bir deneyim olabilir. Dil öğrenme sürecinde kendi öğrenme tarzınızı keşfetmek önemlidir. Dil öğrenmek aynı zamanda farklı kültürleri keşfetmenin harika bir yoludur.
    Bilgisayar oyunları, gençler arasında popüler bir eğlence ve zaman geçirme aktivitesidir. Özellikle rekabetçi oyunlar, oyuncular arasında büyük bir ilgi uyandırmaktadır.
    Doğa yürüyüşleri, stres atmanın ve doğanın güzelliklerini keşfetmenin harika bir yoludur. Yürüyüş sırasında doğal bir ortamda olmak, zihinsel sağlığa olumlu etkiler yapabilir.
    Müzik, insanların duygusal ifadesi için güçlü bir araçtır. Farklı müzik türleri, farklı duygusal durumları yansıtabilir ve dinleyicilere benzersiz bir deneyim sunabilir.
    Bilim kurgu romanları, hayal gücünü genişleten ve alternatif gerçekliklere yol açan ilginç hikayeler sunar. Bu tür romanlar, okuyucuları farklı dünyalara taşıyabilir.
    Egzersiz yapmak, fiziksel sağlığı artırmak ve enerji seviyelerini yükseltmek için etkili bir yöntemdir. Egzersiz yapmak aynı zamanda ruh halini iyileştirebilir ve stresi azaltabilir.
    Gastronomi, farklı kültürlerin mutfağını keşfetmenin keyifli bir yoludur. Yemek yapmak veya yeni restoranlar denemek, lezzetli bir macera olabilir.
    Bilim ve teknoloji, günümüzde hızla ilerleyen alanlardır. Yapay zeka ve uzay keşifleri gibi konular, bilim meraklıları için büyük ilgi çekicilik taşır.
    Sanat, ifade özgürlüğü sağlayan ve estetik deneyimi zenginleştiren bir yoldur. Farklı sanat türleri, insanların duygusal ve yaratıcı yönlerini keşfetmelerine yardımcı olabilir.
    Gönüllü çalışmalar, topluma yardım etmenin ve sosyal sorumluluk almanın önemli bir yolu olabilir. Gönüllü olarak zaman ayırmak, insanlar arasında bağlantı kurma fırsatı sunabilir.
""".split('\n')

# test_corpus = """
#     Bilgisayar😊 oyunları, gençler arasında popüler bir eğlence ve zaman geçirme aktivitesidir. Özellikle rekabetçi oyunlar, oyuncular arasında büyük bir ilgi uyandırmaktadır.
#     Yeni bir dil öğrenmek, zorlu ancak ödüllendirici bir deneyim olabilir. Dil öğrenme sürecinde kendi öğrenme tarzınızı keşfetmek önemlidir. Dil öğrenmek aynı zamanda farklı kültürleri keşfetmenin harika bir yoludur.
#     Doğa yürüyüşleri, stres atmanın ve doğanın güzelliklerini keşfetmenin harika bir yoludur. Yürüyüş sırasında doğal bir ortamda olmak, zihinsel sağlığa olumlu etkiler yapabilir.
#     Müzik, insanların duygusal ifadesi için güçlü bir araçtır. Farklı müzik türleri😊, farklı duygusal durumları yansıtabilir ve dinleyicilere benzersiz bir deneyim sunabilir.
#     Bilim kurgu romanları, hayal gücünü genişleten ve alternatif gerçekliklere yol açan ilginç hikayeler sunar. Bu tür romanlar, okuyucuları farklı dünyalara taşıyabilir.
#     Egzersiz yapmak, fiziksel sağlığı artırmak ve enerji seviyelerini yükseltmek için etkili bir yöntemdir. Egzersiz yapmak aynı zamanda ruh halini iyileştirebilir ve stresi azaltabilir.
#     Gastronomi, farklı kültürlerin mutfağını keşfetmenin keyifli bir yoludur. Yemek yapmak veya yeni restoranlar denemek, lezzetli bir macera olabilir.
#     Bilim ve teknoloji, günümüzde hızla ilerleyen alanlardır. Yapay zeka ve uzay keşifleri gibi konular, bilim meraklıları için büyük ilgi çekicilik taşır.
#     Sanat, ifade özgürlüğü sağlayan ve estetik deneyimi zenginleştiren bir yoldur. Farklı sanat türleri, insanların duygusal ve yaratıcı yönlerini keşfetmelerine yardımcı olabilir.
#     Gönüllü çalışmalar, topluma yardım etmenin ve sosyal sorumluluk almanın önemli bir yolu olabilir. Gönüllü olarak zaman ayırmak, insanlar arasında bağlantı kurma fırsatı sunabilir.
# """.split('\n')

## Initialize

In [2]:
def show_result(_encoder):
    tokenize_res = _encoder.tokenize(test)
    y = _encoder.transform([test])
    x = _encoder.inverse_transform(y)

    print('Text: ')
    print(test)
    print('Tokenize result: ')
    print(tokenize_res)
    print('Transform result: ')
    print(y)
    print('Inverse transform: ')
    print(x)

## Character Level BPE

In [3]:
from collections import Counter
from typing import Dict, Iterable, Callable, List, Any, Iterator
from itertools import chain
from functools import reduce

from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm

import logging
import toolz
import json
import re


class BytePairEncoder:
    EOW = '__eow'
    SOW = '__sow'
    UNK = '__unk'
    PAD = '__pad'

    def __init__(self, vocab_size=1000, log_level=logging.WARNING):
        self._logger = logging.getLogger('BytePairEncoderLogger')
        self._logger.setLevel(log_level)

        self.merges = {}
        self.inverse_merges = {}
        self.vocab = []
        self.inverse_vocab = {}

        self.token_mapper = {
            BytePairEncoder.SOW: '',
            BytePairEncoder.EOW: ' '
        }

        self.required_tokens = [BytePairEncoder.SOW, BytePairEncoder.EOW, BytePairEncoder.UNK]

        self.vocab_size = vocab_size
        self._logger.debug('Initialized')

    def __set_log_level(self, log_level):
        self._logger.setLevel(log_level)

    def __tokenize_word(self, sentence: str):
        return wordpunct_tokenize(sentence)

    def __initialize_word_frequencies(self, corpus: List[str]):
        vocab = {}
        for sentence in corpus:
            for word in self.__tokenize_word(sentence):
                vocab[word] = vocab.get(word, 0) + 1
        self._logger.debug('Word frequency map initialized!')
        return vocab

    def __initialize_base_vocab(self, word_freqs):
        char_freqs = {}
        for word, frequency in word_freqs.items():
            for char in word:
                char_freqs[char] = char_freqs.get(char, 0) + frequency
        char_freqs = list(map(lambda x: x[0], sorted(char_freqs.items(), key=lambda x: x[1], reverse=True)))

        base_vocab = self.required_tokens + char_freqs
        self._logger.debug('Base vocabulary initialized!')
        return base_vocab


    def __compute_pair_freqs(self, word_freqs, splits):
        pair_freqs = {}
        for word, freq in word_freqs.items():
            split = splits[word]
            if len(split) == 1:
                continue

            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] = pair_freqs.get(pair, 0) + freq
        return pair_freqs

    def __get_most_frequent_pair(self, word_freqs, splits):
        pair_freqs =  self.__compute_pair_freqs(word_freqs, splits)
        most_frequent_pair = max(pair_freqs, key=pair_freqs.get)
        return most_frequent_pair

    def __learn_vocab(self, word_freqs, vocab):
        merges = {}
        splits = {word: [c for c in word] for word in word_freqs.keys()}
        idx = len(vocab)
        while len(vocab) < self.vocab_size:
            if all(len(tokens) <= 1 for tokens in splits.values()):
                self._logger.warning('All words are tokenized. There is no pair to merge. Breaking...')
                break

            most_frequent_pair = self.__get_most_frequent_pair(word_freqs, splits)
            (a, b) = most_frequent_pair
            for word in word_freqs:
                split = splits[word]
                if len(split) == 1:
                    continue

                i = 0
                while i < len(split) - 1:
                    if split[i] == a and split[i + 1] == b:
                        split = split[:i] + [a + b] + split[i + 2:]
                    else:
                        i += 1
                splits[word] = split

            merges[most_frequent_pair] = idx
            vocab.append(a + b)
            idx += 1
        self._logger.debug('BPE vocabulary and merge map created!')
        return vocab, merges

    def fit(self, corpus: List[str]):
        word_freqs = self.__initialize_word_frequencies(corpus)
        vocab = self.__initialize_base_vocab(word_freqs)

        self.vocab, self.merges = self.__learn_vocab(word_freqs, vocab)
        self.inverse_vocab = {token: idx for idx, token in enumerate(self.vocab)}
        self.inverse_merges = {idx: pair for pair, idx in self.merges.items()}

    def tokenize(self, text):
        text = self.__tokenize_word(text)
        splits = [[l for l in word] for word in text]

        for pair, merge in self.merges.items():
            for idx, split in enumerate(splits):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [pair[0] + pair[1]] + split[i + 2:]
                    else:
                        i += 1
                splits[idx] = split

        for idx, split in enumerate(splits):
            splits[idx] = [BytePairEncoder.SOW] + splits[idx] + [BytePairEncoder.EOW]

        return sum(splits, [])

    def single_transform(self, text):
        tokens = self.tokenize(text)
        encoded = []
        for token in tokens:
            if token in self.vocab:
                encoded.append(self.inverse_vocab[token])
            else:
                self._logger.debug(f'Character \'{token}\' not found in vocabulary, adding UNK token!')
                encoded.append(self.inverse_vocab[BytePairEncoder.UNK])
        return encoded

    def transform(self, list_of_texts):
        return [self.single_transform(text) for text in list_of_texts]

    def single_inverse_transform(self, tokens):
        decoded = ''
        for idx in tokens:
            token = self.vocab[idx]
            decoded += self.token_mapper.get(token, token)
        return decoded.strip()

    def inverse_transform(self, list_of_tokens):
        return [self.single_inverse_transform(tokens) for tokens in list_of_tokens]

In [4]:
encoder = BytePairEncoder(vocab_size=500, log_level=logging.DEBUG)
encoder.fit(test_corpus)

DEBUG:BytePairEncoderLogger:Initialized
DEBUG:BytePairEncoderLogger:Word frequency map initialized!
DEBUG:BytePairEncoderLogger:Base vocabulary initialized!
DEBUG:BytePairEncoderLogger:BPE vocabulary and merge map created!


In [5]:
test = 'öğrenmesini tamamlayan tokenizer bu metni tokenlerine ayırıyor.'
# test = 'this text is written in a different language'
# test = 'öğrenmesini tamamlayan tokenizer bu metni tokenlerine ayırıyor.😊'
# test = 'Thissrasp is ~noxt😊 a token.'
# test = '学習を完了したトークナイザは、このテキストをトークンに分割します。'

In [6]:
show_result(encoder)

Text: 
öğrenmesini tamamlayan tokenizer bu metni tokenlerine ayırıyor.
Tokenize result: 
['__sow', 'öğren', 'm', 'es', 'in', 'i', '__eow', '__sow', 'ta', 'ma', 'm', 'l', 'ay', 'an', '__eow', '__sow', 't', 'o', 'ken', 'iz', 'er', '__eow', '__sow', 'b', 'u', '__eow', '__sow', 'm', 'et', 'n', 'i', '__eow', '__sow', 't', 'o', 'ken', 'lerin', 'e', '__eow', '__sow', 'ay', 'ır', 'ı', 'y', 'or', '__eow', '__sow', '.', '__eow']
Transform result: 
[[0, 102, 10, 82, 49, 4, 1, 0, 119, 126, 10, 7, 65, 48, 1, 0, 13, 18, 250, 125, 47, 1, 0, 19, 14, 1, 0, 10, 55, 8, 4, 1, 0, 13, 18, 250, 96, 5, 1, 0, 65, 67, 12, 11, 103, 1, 0, 21, 1]]
Inverse transform: 
['öğrenmesini tamamlayan tokenizer bu metni tokenlerine ayırıyor .']


## Byte Level BPE

In [7]:
from collections import Counter
from typing import Dict, Iterable, Callable, List, Any, Iterator
from itertools import chain
from functools import reduce

from nltk.tokenize import wordpunct_tokenize
from tqdm import tqdm

import logging
import toolz
import json
import re

from time import sleep


class ByteLevelBytePairEncoding:

    def __init__(self, vocab_size=1000, log_level=logging.WARNING):
        self._logger = logging.getLogger('BytePairEncoderLogger')
        self._logger.setLevel(log_level)

        self.merges = {}
        self.inverse_merges = {}
        self.vocab = []
        self.inverse_vocab = {}

        self.EOW = 256
        self.SOW = 257
        self.UNK = 258
        self.PAD = 259

        self.token_mapper = {
            self.EOW: 32 # Space
        }

        self.required_tokens = [self.SOW, self.EOW, self.UNK]

        self.vocab_size = vocab_size
        self._logger.debug('Initialized')

    def __set_log_level(self, log_level):
        self._logger.setLevel(log_level)

    def __tokenize_word(self, sentence: str):
        return [''.join(word).encode('utf-8') for word in wordpunct_tokenize(sentence)]

    def __initialize_word_frequencies(self, corpus: List[str]):
        vocab = {}
        for sentence in corpus:
            for word in self.__tokenize_word(sentence):
                vocab[word] = vocab.get(word, 0) + 1
        self._logger.debug('Word frequency map initialized!')
        return vocab

    def __initialize_base_vocab(self, word_freqs):
        byte_freqs = {}
        for word, frequency in word_freqs.items():
            for char in word:
                byte_freqs[char] = byte_freqs.get(char, 0) + frequency
        byte_freqs = list(map(lambda x: x[0], sorted(byte_freqs.items(), key=lambda x: x[1], reverse=True)))
        base_vocab = list(dict.fromkeys(self.required_tokens[:] + byte_freqs + list(range(256))))

        self._logger.debug('Base vocabulary initialized!')
        return base_vocab


    def __compute_pair_freqs(self, word_freqs, splits):
        pair_freqs = {}
        for word, freq in word_freqs.items():
            split = splits[word]
            if len(split) == 1:
                continue

            for i in range(len(split) - 1):
                pair = (split[i], split[i + 1])
                pair_freqs[pair] = pair_freqs.get(pair, 0) + freq
        return pair_freqs

    def __get_most_frequent_pair(self, word_freqs, splits):
        pair_freqs =  self.__compute_pair_freqs(word_freqs, splits)
        most_frequent_pair = max(pair_freqs, key=pair_freqs.get)
        return most_frequent_pair

    def __learn_vocab(self, word_freqs, vocab):
        merges = {}
        splits = {word: [c for c in word] for word in word_freqs.keys()}
        new_token_count = 0
        start_idx = len(vocab)
        while len(vocab) < self.vocab_size:
            if all(len(tokens) <= 1 for tokens in splits.values()):
                self._logger.warning('All words are tokenized. There is no pair to merge. Breaking...')
                break

            most_frequent_pair = self.__get_most_frequent_pair(word_freqs, splits)
            (a, b) = most_frequent_pair
            new_token_count += 1
            for word in word_freqs:
                split = splits[word]
                if len(split) == 1:
                    continue

                i = 0
                while i < len(split) - 1:
                    if split[i] == a and split[i + 1] == b:
                        split = split[:i] + [start_idx + new_token_count] + split[i + 2:]
                    else:
                        i += 1
                splits[word] = split

            merges[most_frequent_pair] = start_idx + new_token_count
            vocab.append(start_idx + new_token_count)
        self._logger.debug('BPE vocabulary and merge map created!')
        return vocab, merges

    def fit(self, corpus: List[str]):
        word_freqs = self.__initialize_word_frequencies(corpus)
        vocab = self.__initialize_base_vocab(word_freqs)
        self.vocab, self.merges = self.__learn_vocab(word_freqs, vocab)

        self.inverse_vocab = {token: idx for idx, token in enumerate(self.vocab)}
        self.inverse_merges = {idx: pair for pair, idx in self.merges.items()}

    def tokenize(self, text):
        text = self.__tokenize_word(text)
        splits = [[l for l in word] for word in text]

        for pair, merge in self.merges.items():
            for idx, split in enumerate(splits):
                i = 0
                while i < len(split) - 1:
                    if split[i] == pair[0] and split[i + 1] == pair[1]:
                        split = split[:i] + [merge] + split[i + 2:]
                    else:
                        i += 1
                splits[idx] = split

        for idx, split in enumerate(splits):
            splits[idx] = [self.SOW] + splits[idx] + [self.EOW]
        return sum(splits, [])

    def transform(self, text_list):
        # for text in text_list:
        #     yield self.tokenize(text)
        return [self.tokenize(text) for text in text_list]

    def _inverse_transform_single(self, tokens):
        idx = 0
        decoded = []
        while idx < len(tokens) -1:
            token = tokens[idx]
            if token in self.inverse_merges:
                merges = self.inverse_merges[token]
                tokens = tokens[:idx] + [merges[0], merges[1]] + tokens[idx + 1:]
            else:
                idx += 1
                if token in [self.SOW, self.UNK]:
                    continue
                token = self.token_mapper.get(token, token)
                decoded.append(token)
        return bytes(decoded).decode('utf-8')

    def inverse_transform(self, token_lists):
        # for tokens in token_lists:
        #     yield self._inverse_transform_single(tokens)
        return [self._inverse_transform_single(tokens) for tokens in token_lists]

In [8]:
byte_level_encoder = ByteLevelBytePairEncoding(vocab_size=500, log_level=logging.DEBUG)
byte_level_encoder.fit(test_corpus)

DEBUG:BytePairEncoderLogger:Initialized
DEBUG:BytePairEncoderLogger:Word frequency map initialized!
DEBUG:BytePairEncoderLogger:Base vocabulary initialized!
DEBUG:BytePairEncoderLogger:BPE vocabulary and merge map created!


In [9]:
# test = 'öğrenmesini tamamlayan tokenizer bu metni tokenlerine ayırıyor.'
# test = 'this text is written in a different language'
# test = 'öğrenmesini tamamlayan tokenizer bu metni tokenlerine ayırıyor.😊'
# test = 'Thissrasp is ~noxt😊 a token.'
test = '学習を完了したトークナイザは、このテキストをトークンに分割します。'

In [10]:
show_result(byte_level_encoder)

Text: 
学習を完了したトークナイザは、このテキストをトークンに分割します。
Tokenize result: 
[257, 229, 173, 166, 231, 191, 146, 227, 130, 146, 229, 174, 140, 228, 186, 134, 227, 129, 151, 227, 129, 159, 227, 131, 136, 227, 131, 188, 227, 130, 175, 227, 131, 138, 227, 130, 164, 227, 130, 182, 227, 129, 175, 256, 257, 227, 128, 129, 256, 257, 227, 129, 147, 227, 129, 174, 227, 131, 134, 227, 130, 173, 227, 130, 185, 227, 131, 136, 227, 130, 146, 227, 131, 136, 227, 131, 188, 227, 130, 175, 227, 131, 179, 227, 129, 171, 229, 136, 134, 229, 137, 178, 227, 129, 151, 227, 129, 190, 227, 129, 153, 256, 257, 227, 128, 130, 256]
Transform result: 
[[257, 229, 173, 166, 231, 191, 146, 227, 130, 146, 229, 174, 140, 228, 186, 134, 227, 129, 151, 227, 129, 159, 227, 131, 136, 227, 131, 188, 227, 130, 175, 227, 131, 138, 227, 130, 164, 227, 130, 182, 227, 129, 175, 256, 257, 227, 128, 129, 256, 257, 227, 129, 147, 227, 129, 174, 227, 131, 134, 227, 130, 173, 227, 130, 185, 227, 131, 136, 227, 130, 146, 227, 131, 136, 227, 131, 188,